<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/american_boundary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize


def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def American(S, K, T, sigma, r, q, N, kind):
    bs = callBS if kind == "call" else putBS
    intrinsic = lambda x: (x - K if kind == "call" else K - x)
    dt = T / N
    up = np.exp(sigma * np.sqrt(dt))
    down = 1 / up
    prob = (np.exp((r - q) * dt) - down) / (up - down)
    discount = np.exp(-r * dt)

    # Black-Scholes at penultimate date
    x = S * up ** np.arange(N - 1, -N - 1, -2)
    v = bs(x, K, T, sigma, r, q)
    for n in range(N - 2, -1, -1):
        x = S * up ** n
        v[0] = np.maximum(intrinsic(x), discount * (prob * v[0] + (1 - prob) * v[1]))
        for i in range(1, n + 1):
            x *= down * down
            v[i] = np.maximum(
                intrinsic(x), discount * (prob * v[i] + (1 - prob) * v[i + 1])
            )
    return v[0]


def Richardson(S, K, T, sigma, r, q, N, kind):
    y1 = American(S, K, T, sigma, r, q, N, kind)
    y2 = American(S, K, T, sigma, r, q, 2 * N, kind)
    return 2 * y2 - y1


def Boundary(K, times, sigma, r, q, kind, N=30):
    intrinsic = lambda x: (x - K if kind == "call" else K - x)
    richardson = lambda x: Richardson(x, K, T, sigma, r, q, N, kind)
    bdys = []

    for t in times[:-1]:

        def constraint(x):
            return Richardson(x, K, times[-1] - t, sigma, r, q, N, kind) - intrinsic(x)

        def objective(x):
            return x if kind == "call" else -x

        cons = {"type": "eq", "fun": constraint}
        b = minimize(objective, x0=K, method="SLSQP", constraints=cons).x[0]
        bdys.append(b)

    bdys.append(K)
    return bdys



# an example option

K = 50
T = 1
sigma = 0.4
r = 0.02
q = 0.03
kind = 'call'

# for efficiency, we only compute the boundary for these grids
times = T * np.array([0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.98, 1])

boundaries = Boundary(K, times, sigma, r, q, kind)
pd.DataFrame([times, boundaries]).transpose().rename(columns={0 : 'Time (Year)', 1 : 'Excercise Boundary (Exercise when Above It)'})



,Time (Year),Excercise Boundary (Exercise when Above It)
0,0.00,116.794186
1,0.25,109.239041
2,0.50,99.562310
3,0.75,86.511716
4,0.90,73.938831
5,0.95,67.420205
6,0.98,61.390634
7,1.00,50.000000
